In [20]:
import os
import pandas as pd
from openpyxl import load_workbook

In [4]:
os.listdir()

['.git',
 'first notebook.ipynb',
 'pisa_data_to_df.ipynb',
 'Reading Material',
 'README.md']

In [9]:
# Specify the directory you want to list
folder_path = 'Reading Material/Data - Sheets'

# List all files in the specified directory
file_list = os.listdir(folder_path)

file_list

['EDU-2019-4228-EN-T010.XLSX',
 'EDU-2019-4228-EN-T014.XLSX',
 'EDU-2019-4229-EN-T007.XLSX',
 'EDU-2019-4229-EN-T008.XLSX',
 'EDU-2019-4229-EN-T009.XLSX',
 'EDU-2019-4229-EN-T010.XLSX',
 'EDU-2019-4229-EN-T011.XLSX',
 'EDU-2019-4229-EN-T012.XLSX',
 'EDU-2019-4229-EN-T013.XLSX',
 'EDU-2019-4229-EN-T014.XLSX',
 'EDU-2019-4230-EN-T006.XLSX',
 'EDU-2019-4230-EN-T008.XLSX',
 'EDU-2019-4230-EN-T009.XLSX',
 'EDU-2019-4230-EN-T010.XLSX',
 'EDU-2019-4230-EN-T011.XLSX',
 'EDU-2019-4230-EN-T012.XLSX',
 'EDU-2019-4230-EN-T013.XLSX',
 'EDU-2019-4230-EN-T015.XLSX',
 'EDU-2019-4230-EN-T016.XLSX',
 'EDU-2019-4230-EN-T017.XLSX',
 'EDU-2019-4230-EN-T018.XLSX',
 'EDU-2020-485-EN-T009.XLSX']

In [28]:
# Load the Excel file
file_path = 'Reading Material/Data - Sheets/EDU-2019-4228-EN-T010.XLSX'

# Attempt to read all sheets to find out their names
try:
    # This will not read any data, just the sheet names
    all_sheets = pd.ExcelFile(file_path).sheet_names
except Exception as e:
    all_sheets = str(e)

all_sheets

['TOC',
 'Table I.A2.1',
 'Table I.A2.2',
 'Table I.A2.3',
 'Table I.A2.4',
 'Table I.A2.5',
 'Table I.A2.6',
 'Table I.A2.7',
 'Table I.A2.8',
 'Table I.A2.9',
 'Table I.A2.10',
 'Table I.A2.11',
 'Table I.A2.12',
 'Table I.A2.13',
 'Table I.A2.14',
 'Table I.A2.15']

In [39]:
# Read the specified sheet into a dataframe, assuming header starts from row 14 (index 13 in zero-based indexing)
sheet_name = 'Table I.A2.1'

# Load the workbook and select the worksheet
workbook = load_workbook(file_path, data_only=True)
sheet = workbook[sheet_name]


# Define the header rows based on the provided screenshot
category_title_row_index = 11  # Excel row 12
column_title_row_index = 12    # Excel row 13
column_number_row_index = 13   # Excel row 14


# Read the category and column titles from the Excel file
category_titles_row = pd.read_excel(file_path, sheet_name=sheet_name, nrows=1, skiprows=category_title_row_index-1).values[0]
column_titles_row = pd.read_excel(file_path, sheet_name=sheet_name, nrows=1, skiprows=column_title_row_index-1).values[0]


# Propagate the category titles to account for merged cells
propagated_category_titles = []
for title in category_titles_row:
    if pd.isnull(title) and propagated_category_titles:
        title = propagated_category_titles[-1]  # Use the previous title if current is NaN
    propagated_category_titles.append(title)

propagated_category_titles

# Combine the category and column titles
combined_header = []
for category, column in zip(propagated_category_titles, column_titles_row):
    category = category if category is not None else ''
    column = column if column is not None else ''
    combined_header.append(f"{category}_{column}".lower().replace(' ', '_'))



['nan_nan',
 'population_and_sample_information_total_population_of_15-year-olds',
 'population_and_sample_information_total_enrolled_population_of_15-year-olds_at_grade_7_or_above',
 'population_and_sample_information_total_in_national_desired_target_population',
 'population_and_sample_information_total_school-level_exclusions',
 'population_and_sample_information_total_in_national_desired_target_population_after_all_school_exclusions_and_before_within-school_exclusions',
 'population_and_sample_information_school-level_exclusion_rate_(%)',
 'population_and_sample_information_number_of_participating_students',
 'population_and_sample_information_weighted_number_of_participating_students',
 'population_and_sample_information_number_of_excluded_students',
 'population_and_sample_information_weighted_number_of_excluded_students',
 'population_and_sample_information_within-school_exclusion_rate_(%)',
 'population_and_sample_information_overall_exclusion_rate_(%)',
 'coverage_indices_cove

In [37]:
column_titles_row

array([nan, 'Total population of 15-year-olds',
       'Total enrolled population of 15-year-olds at grade 7 or above',
       'Total in national desired target population',
       'Total school-level exclusions',
       'Total in national desired target population after all school exclusions and before within-school exclusions',
       'School-level exclusion rate (%)',
       'Number of participating students',
       'Weighted number of participating students',
       'Number of excluded students',
       'Weighted number of excluded students',
       'Within-school exclusion rate (%)', 'Overall exclusion rate (%)',
       'Coverage Index 1: Coverage of national desired population',
       'Coverage Index 2: Coverage of national enrolled population',
       'Coverage Index 3: Coverage of 15-year-old population'],
      dtype=object)

In [36]:
# Sanitize the header names by removing leading and trailing underscores and replacing '__' with '_'
sanitized_header = [header.strip('_').replace('__', '_') for header in combined_header]

# Read the actual data into a DataFrame, skipping the header rows
df_data = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=column_number_row_index)

# Set the correct header for the dataframe, trimming to match the number of data columns
df_data.columns = sanitized_header[:len(df_data.columns)]

# Replace 'm' with np.nan for missing values
df_data.replace('m', pd.NA, inplace=True)

# Display the first few rows of the dataframe
df_data.head()

ValueError: Length mismatch: Expected axis has 16 elements, new values have 14 elements